In [1]:
import otf

In [2]:
v = ([], [])

Otf has its own serialization library.

In [3]:
otf.dumps(v)

'tuple([[], []])'

We support shared references:

In [4]:
v = []
otf.dumps([v, 2, v])

'[[], 2, ref(2)]'

## Shared references
Out of the box, not many types are supported

It's pretty easy to add support for new types. You need to write a "reduction" function that returns three values
+ A callable *fn*
+ A tuple *args* of values that are serialisable
+ A dictionary *kwargs* where the keys are valid python identifiers and the values are serialisable

In [5]:
@otf.register
def _reduce_complex(c: complex):
    return complex, (), {"real": c.real, "imag": c.imag}

In [6]:
otf.dumps(complex(5))

'complex(real=5.0, imag=0.0)'

## Executable format

For cases where people might want to edit serialized values we even offer a format that can be ran in python

In [7]:
c = complex(5, 6)
d = complex(6, 5)
print(otf.dumps({c: d, d: c}, format=otf.EXECUTABLE))

_1 = complex(real=5.0, imag=6.0)

_0 = complex(real=6.0, imag=5.0)

{_1: _0, _0: _1}



Even though this format yields runnable python stubs we do not load it by executing it:

In [8]:
otf.loads(
    """
_1 = complex(real=5.0, imag=6.0)

_0 = complex(real=6.0, imag=5.0)

# This will cause an error when evaluated
_2 = 1 / 0.

{_1: _0, _0: _1}

"""
)

{(5+6j): (6+5j), (6+5j): (5+6j)}

## Producers and reducer

You can convert between any of our format without going through an intermediate representation. This has the advange that it lets you edit any syntatically valid value:

In [9]:
problematic_value = "{'stdin': my_lib.file_descriptor(0), 'stdout': my_lib.file_descriptor(1), 'stderr': ref(3)}"

print(
    otf.pack.text.reduce(
        problematic_value, acc=otf.pack.text.ExecutablePrinter()
    )
)

# There were errors trying to import the following constructors
#
# + 'my_lib.file_descriptor': ImportError Failed to find object

_0 = my_lib.file_descriptor(1)

{'stdin': my_lib.file_descriptor(0), 'stdout': _0, 'stderr': _0}



Maybe you switch from my_lib to fd_lib and the format of the file descriptor has changed:

In [10]:
fixed_value = """
# Fixed by till to run with fd_lib

STDIN = fd_lib.fd(1, mode='w')
STDOUT = fd_lib.fd(1, mode='w')
STDERR = STDOUT

{'stdin': STDIN, 'stdout': STDOUT, 'stderr': STDERR}
"""

print(otf.pack.text.reduce(fixed_value, acc=otf.pack.text.CompactPrinter()))

{'stdin': fd_lib.fd(1, mode='w'), 'stdout': fd_lib.fd(1, mode='w'), 'stderr': ref(4)}


Note that the size of some of the nodes in the tree changed. Had you edited the "CompactPrinter" form of the document you would have had to adjust that number in the ref